# 실습: Azure AI Search와 OpenAI 임베딩을 활용한 RAG 구현


## 왜 RAG(Retrieval Augmented Generation)가 필요한가?


기존의 대형 언어 모델(LLM)은 사전 학습된 지식만을 바탕으로 답변을 생성하기 때문에, 최신 정보나 특정 도메인 데이터에 대한 답변이 부정확하거나 제한적일 수 있습니다. RAG는 외부 데이터베이스(예: 사내 문서, 최신 자료 등)에서 관련 정보를 검색(Retrieval)한 뒤, 이를 LLM의 입력 컨텍스트로 활용하여 보다 정확하고 신뢰할 수 있는 답변을 생성(Generation)하는 방식입니다.


- **최신성**: LLM이 알지 못하는 최신 정보나 사내 데이터도 답변에 반영할 수 있습니다.
- **정확성**: 검색된 실제 문서를 근거로 답변을 생성하므로, 신뢰도와 근거 기반 답변이 강화됩니다.
- **확장성**: 다양한 데이터 소스와 결합하여, 특정 도메인이나 업무에 특화된 AI 서비스를 쉽게 구축할 수 있습니다.


이 노트북에서는 JSON 데이터를 Azure AI Search에 업로드하고, Azure OpenAI의 임베딩 API를 활용해 벡터 인덱싱을 수행한 뒤, 이를 기반으로 간단한 RAG를 구현하는 과정을 단계별로 실습합니다.


- Azure AI Search 및 OpenAI 리소스 준비
- JSON 데이터 업로드 및 인덱스 생성
- 임베딩 생성 및 저장
- 자연어 쿼리로 검색
- 간단한 RAG 구현


## 1. Azure 리소스 생성

이 실습을 위해서는 다음과 같은 Azure 리소스가 필요합니다. 각 리소스는 RAG (Retrieval Augmented Generation) 파이프라인의 특정 단계를 수행하는 데 사용됩니다.

1.  **Azure Blob Storage 계정 및 컨테이너**:
    *   **목적**: 원본 데이터(이 실습에서는 `sample.json` 파일)를 저장합니다. Azure AI Search가 이 데이터를 읽어 인덱싱을 수행합니다.
    *   **필요성**: 대량의 데이터를 안정적으로 저장하고, 다른 Azure 서비스에서 쉽게 접근할 수 있도록 하기 위함입니다.

2.  **Azure AI Search 서비스**:
    *   **목적**: 업로드된 JSON 데이터에 대한 검색 인덱스를 생성하고 관리합니다. 특히, 텍스트 데이터로부터 생성된 벡터 임베딩을 저장하고, 이를 기반으로 유사도 검색(벡터 검색)을 수행하는 데 핵심적인 역할을 합니다.
    *   **필요성**: RAG의 'Retrieval' 단계를 담당하여, 사용자 질문과 관련된 정보를 효율적으로 찾아내기 위함입니다.

3.  **Azure OpenAI 서비스**:
    *   **목적**:
        *   **임베딩 모델 배포 (예: `text-embedding-3-large`)**: JSON 데이터의 텍스트 내용을 숫자 벡터(임베딩)로 변환하여 Azure AI Search 인덱스에 저장합니다. 또한, 사용자 질문도 임베딩으로 변환하여 검색에 사용합니다.
        *   **채팅 모델 배포 (예: `gpt-4.1-mini`, `gpt-4o-mini`)**: Azure AI Search를 통해 검색된 관련 문서를 컨텍스트로 활용하여, 사용자의 질문에 대한 최종 답변을 생성합니다 (RAG의 'Generation' 단계).
    *   **필요성**: 데이터의 의미를 이해하고(임베딩), 검색된 정보를 바탕으로 자연스러운 답변을 생성하기(LLM) 위함입니다.

**Azure 리소스 생성 참고:**
1. [Azure Blob Storage 계정 및 컨테이너 생성 가이드](https://learn.microsoft.com/azure/storage/blobs/storage-quickstart-blobs-portal)
2. [Azure AI Search 서비스 생성 가이드](https://learn.microsoft.com/azure/search/search-create-service-portal)
3. [Azure OpenAI 서비스 리소스 생성 및 모델 배포 가이드](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal) (이 실습에서는 임베딩 모델과 채팅 모델 두 가지를 배포해야 합니다.)

> 참고: [Azure AI Search 공식 문서](https://learn.microsoft.com/azure/search/search-what-is-azure-search)

### [중요] Azure 리소스 변수 선언

아래 코드셀에서는 Azure 리소스(리소스 그룹, 위치, 스토리지 계정, 컨테이너, AI Search 서비스 등)의 이름과 위치를 변수로 선언합니다. 

- **목적:**
    - CLI 명령어 실행 시 반복적으로 사용할 값(리소스명, 지역 등)을 변수로 지정하여, 실습 과정에서 일관성 있게 활용할 수 있도록 합니다.
    - 변수만 수정하면 이후 셀의 명령어들이 자동으로 해당 값으로 반영되어 편리합니다.
- **사용 방법:**
    - 실습 환경에 맞게 변수값(예: 리소스 그룹명, 지역, 스토리지 계정명 등)을 수정하세요.
    - 이후 셀에서 `$변수명` 형태로 참조되어 CLI 명령어에 자동으로 적용됩니다.
- **예시:**
    - `RESOURCE_GROUP="myResourceGroup"` → 실제 본인이 생성한 리소스 그룹명으로 변경
    - `LOCATION="koreacentral"` → 원하는 Azure 지역으로 변경

> ⚠️ 변수값을 잘못 입력하면 리소스 생성이나 명령어 실행이 실패할 수 있으니, Azure Portal에서 실제 리소스명을 확인 후 입력하세요.


In [1]:
# Azure 리소스 변수 선언 (실행 전 변수값을 원하는 값으로 수정하세요)
RESOURCE_GROUP="myResourceGroup"
LOCATION="koreacentral"
STORAGE_ACCOUNT="myraglabstorage9556"
CONTAINER_NAME="json-container"
SEARCH_SERVICE="my-rag-lab-search"

> ⚠️ **참고:** 3번 노트북(03-deploying-to-aks-using-docker-and-acr.ipynb)에서 이미 리소스 그룹을 생성했다면, 아래 셀은 실행하지 않고 스킵합니다.

In [ ]:
# 1. 리소스 그룹 생성 (필수 단계 아님)
!az group create --name $RESOURCE_GROUP --location $LOCATION

In [2]:
# 2-1. Blob Storage 계정 생성
!az storage account create \
  --name $STORAGE_ACCOUNT \
  --resource-group $RESOURCE_GROUP \
  --location $LOCATION \
  --sku Standard_LRS

{| Finished ..
  "accessTier": "Hot",
  "accountMigrationInProgress": null,
  "allowBlobPublicAccess": false,
  "allowCrossTenantReplication": false,
  "allowSharedKeyAccess": null,
  "allowedCopyScope": null,
  "azureFilesIdentityBasedAuthentication": null,
  "blobRestoreStatus": null,
  "creationTime": "2025-05-15T00:26:03.086819+00:00",
  "customDomain": null,
  "defaultToOAuthAuthentication": null,
  "dnsEndpointType": null,
  "enableExtendedGroups": null,
  "enableHttpsTrafficOnly": true,
  "enableNfsV3": null,
  "encryption": {
    "encryptionIdentity": null,
    "keySource": "Microsoft.Storage",
    "keyVaultProperties": null,
    "requireInfrastructureEncryption": null,
    "services": {
      "blob": {
        "enabled": true,
        "keyType": "Account",
        "lastEnabledTime": "2025-05-15T00:26:03.180586+00:00"
      },
      "file": {
        "enabled": true,
        "keyType": "Account",
        "lastEnabledTime": "2025-05-15T00:26:03.180586+00:00"
      },
      "queu

In [3]:
# 2-2. Blob Storage 컨테이너 생성 (Azure AD 권한 사용)
!az storage container create \
  --account-name $STORAGE_ACCOUNT \
  --name $CONTAINER_NAME \
  --auth-mode login

{
  "created": true
}


In [ ]:
# 연결 문자열 확인 (환경 변수에 사용)
!az storage account show-connection-string \
  --name $STORAGE_ACCOUNT \
  --resource-group $RESOURCE_GROUP \
  --query connectionString --output tsv

In [5]:
# 3. Azure AI Search 서비스 생성
!az search service create \
  --name $SEARCH_SERVICE \
  --resource-group $RESOURCE_GROUP \
  --location $LOCATION \
  --sku basic

{- Finished ..
  "authOptions": {
    "apiKeyOnly": {}
  },
  "disableLocalAuth": false,
  "encryptionWithCmk": {
    "encryptionComplianceStatus": "Compliant",
    "enforcement": "Unspecified"
  },
  "hostingMode": "default",
  "id": "/subscriptions/49a89096-a0ae-4e59-816b-dcb0a6fe9168/resourceGroups/myResourceGroup/providers/Microsoft.Search/searchServices/my-rag-lab-search",
  "location": "Korea Central",
  "name": "my-rag-lab-search",
  "networkRuleSet": {
    "ipRules": []
  },
  "partitionCount": 1,
  "privateEndpointConnections": [],
  "provisioningState": "succeeded",
  "publicNetworkAccess": "Enabled",
  "replicaCount": 1,
  "resourceGroup": "myResourceGroup",
  "semanticSearch": "free",
  "sharedPrivateLinkResources": [],
  "sku": {
    "name": "basic"
  },
  "status": "running",
  "statusDetails": "",
  "tags": {},
  "type": "Microsoft.Search/searchServices"
}


### [중요] `.env` 파일에 이 노트북을 위한 추가 환경 변수 설정

`README.md` 파일에 `.env` 파일 생성 및 Azure OpenAI 관련 기본 환경 변수 설정에 대한 안내가 이미 포함되어 있습니다. 이 노트북을 실행하기 위해서는 해당 기본 설정 외에 Azure AI Search 및 Azure Blob Storage 관련 환경 변수를 `.env` 파일에 추가해야 합니다.

**이 노트북에 필요한 추가 환경 변수 예시:**

```env
# --- README.md에 명시된 Azure OpenAI 변수들 ---

# AZURE_OPENAI_ENDPOINT="your_openai_endpoint_here"
# AZURE_OPENAI_API_KEY="your_openai_api_key_here"
# AZURE_OPENAI_API_VERSION="2024-02-01"
# AZURE_OPENAI_DEPLOYMENT_EMBEDDING_NAME="your_embedding_deployment_name_here"
# AZURE_OPENAI_DEPLOYMENT_NAME="your_chat_deployment_name_here"

# --- 이 노트북을 위한 추가 변수 ---

# Azure AI Search
AZURE_SEARCH_ENDPOINT="your_search_endpoint_here"
AZURE_SEARCH_API_KEY="your_search_api_key_here"
AZURE_SEARCH_INDEX_NAME="your_search_index_name_here"

# Azure Blob Storage
AZURE_BLOB_CONNECTION_STRING="your_blob_connection_string_here"
```

*   **확인:** `README.md`의 안내에 따라 Azure OpenAI 관련 환경 변수(`AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_API_KEY` 등)가 이미 `.env` 파일에 설정되어 있는지 확인하세요.
*   위 목록에서 `#`으로 시작하는 줄은 주석이며, 실제 값으로 `"your_..._here"` 부분을 교체해야 합니다.
*   `python-dotenv` 라이브러리를 사용하여 이 변수들을 로드하는 방법은 `README.md` 또는 이 노트북의 다른 Python 셀들을 참고하세요. 대부분의 코드 셀 시작 부분에 `load_dotenv()`가 이미 포함되어 있습니다.


### Azure AI Search의 Endpoint와 API Key 확인 방법

- **AZURE_SEARCH_ENDPOINT**: Azure Portal에서 생성한 Azure AI Search 리소스의 개요(Overview) 페이지에서 'URL' 항목을 확인할 수 있습니다. 예시: `https://<search-service-name>.search.windows.net`
- **AZURE_SEARCH_API_KEY**: Azure Portal에서 해당 Search 리소스 > '키(Key)' 메뉴에서 '관리 키(Admin keys)'를 확인할 수 있습니다.

또는 아래 CLI 명령어로도 확인할 수 있습니다:

```bash
# 관리 키 확인
az search admin-key show --service-name <SEARCH_SERVICE> --resource-group <RESOURCE_GROUP>
```


### [중요] 패키지 설치 및 커널 재시작 안내

- 아래 셀을 실행하여 필요한 라이브러리를 설치하세요.
- 설치가 완료된 후에는 **커널을 반드시 재시작**해야 모든 패키지가 정상적으로 인식됩니다.
- 커널 재시작 방법: 메뉴에서 'Runtime' > 'Restart Kernel' 또는 상단의 재시작 아이콘 클릭

In [ ]:
# 필요한 파이썬 패키지 설치
%pip install -r requirements.txt


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2. JSON 데이터 준비 및 업로드

1. JSON 파일을 준비합니다. (예: 각 문서가 하나의 JSON 오브젝트로 구성된 리스트)
2. Azure Blob Storage에 JSON 파일을 업로드합니다.

아래 코드는 JSON 파일을 Azure Blob Storage에 업로드하는 예시입니다.

In [1]:
import os
import json
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

load_dotenv()

blob_conn_str = os.getenv("AZURE_BLOB_CONNECTION_STRING")
container_name = "json-container"
json_path = "./sample.json"  # 실습용 JSON 파일 경로

if not blob_conn_str:
    raise ValueError("AZURE_BLOB_CONNECTION_STRING 환경변수가 설정되지 않았습니다.")

try:
    blob_service_client = BlobServiceClient.from_connection_string(blob_conn_str)
    container_client = blob_service_client.get_container_client(container_name)
    # 컨테이너가 없으면 생성
    try:
        container_client.create_container()
    except Exception:
        pass  # 이미 있으면 무시

    with open(json_path, "rb") as data:
        container_client.upload_blob(name="sample.json", data=data, overwrite=True)
    print("JSON 파일 업로드 성공")
except Exception as e:
    print(f"JSON 파일 업로드 실패: {e}")

JSON 파일 업로드 성공


## 3. Azure AI Search 인덱스 생성

1. Azure AI Search 인덱스 스키마(필드: id, content, metadata, embedding 등)를 정의합니다.
2. Azure AI Search에 인덱스를 생성합니다.

아래 코드는 Azure AI Search에 인덱스를 생성하는 예시입니다.

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters
)

search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_EMBEDDING_NAME")
deployment_model = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
embedding_model = os.getenv("AZURE_OPENAI_DEPLOYMENT_EMBEDDING_NAME")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")

index_client = SearchIndexClient(endpoint=search_endpoint, credential=AzureKeyCredential(search_key))

# 벡터 검색 프로필 및 알고리즘 구성
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(name="myHnsw")
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer_name="myVectorizer"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            vectorizer_name="myVectorizer",
            parameters=AzureOpenAIVectorizerParameters(
                resource_url=azure_openai_endpoint,
                deployment_name=azure_openai_embedding_deployment,
                model_name=embedding_model,
                api_key=azure_openai_key
            )
        )
    ]
)

fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SimpleField(name="content", type=SearchFieldDataType.String, searchable=True),
    SimpleField(name="metadata", type=SearchFieldDataType.String, filterable=True),
    SearchField(
        name="embedding",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=3072,  # text-embedding-3-large 차원수
        vector_search_profile_name="myHnswProfile"
    ),
]

index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)
try:
    index_client.create_or_update_index(index)
    print(f"인덱스 생성 완료: {index_name}")
except Exception as e:
    print(f"인덱스 생성 오류 또는 이미 존재: {e}")

인덱스 생성 완료: json-index


## 4. JSON 데이터 인덱싱

1. JSON에서 텍스트(content 필드 등)를 추출합니다.
2. 추출한 텍스트로 임베딩을 생성합니다.
3. 텍스트와 임베딩을 Azure AI Search 인덱스에 저장합니다.

아래 코드는 Azure Blob Storage와 Search SDK를 활용한 전체 예시입니다.

> **참고:** 벡터 검색을 위해 embedding 필드를 `Collection(SearchFieldDataType.Single)` 타입으로 추가해야 하며, JSON에서 텍스트를 추출한 후 임베딩을 생성하여 인덱스에 저장하는 전체 과정을 코드로 제공합니다.

In [3]:
from openai import AzureOpenAI
from azure.search.documents import SearchClient

openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
embedding_model = os.getenv("AZURE_OPENAI_DEPLOYMENT_EMBEDDING_NAME", "text-embedding-3-large")

search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")

# JSON 데이터 로드 (리스트 형태로 파싱)
json_docs = []
with open(json_path, "r", encoding="utf-8") as f:
    try:
        json_docs = json.load(f)
    except Exception as e:
        print(f"JSON 파일 파싱 오류: {e}")

client = AzureOpenAI(
    api_key=openai_key,
    api_version=openai_api_version,
    azure_endpoint=openai_endpoint
)
search_client = SearchClient(
    endpoint=search_endpoint, 
    index_name=index_name, 
    credential=AzureKeyCredential(search_key)
)

for doc in json_docs:
    content = doc.get("content", "")
    metadata = doc.get("metadata", "")
    doc_id = doc.get("id", None) or str(hash(content))
    embedding_response = client.embeddings.create(
        input=content,
        model=embedding_model
    )
    embedding = embedding_response.data[0].embedding
    index_doc = {"id": doc_id, "content": content, "metadata": metadata, "embedding": embedding}
    try:
        search_client.upload_documents(documents=[index_doc])
        print(f"문서 업로드 완료: {doc_id}")
    except Exception as e:
        print(f"업로드 오류: {e}")

문서 업로드 완료: uniqlo001
문서 업로드 완료: uniqlo002
문서 업로드 완료: uniqlo003
문서 업로드 완료: uniqlo004
문서 업로드 완료: uniqlo005
문서 업로드 완료: uniqlo006
문서 업로드 완료: uniqlo007
문서 업로드 완료: uniqlo008
문서 업로드 완료: uniqlo009
문서 업로드 완료: uniqlo010
문서 업로드 완료: uniqlo011
문서 업로드 완료: uniqlo012
문서 업로드 완료: uniqlo013
문서 업로드 완료: uniqlo014
문서 업로드 완료: uniqlo015
문서 업로드 완료: uniqlo016
문서 업로드 완료: uniqlo017
문서 업로드 완료: uniqlo018
문서 업로드 완료: uniqlo019
문서 업로드 완료: uniqlo020
문서 업로드 완료: uniqlo021
문서 업로드 완료: uniqlo022
문서 업로드 완료: uniqlo023
문서 업로드 완료: uniqlo024
문서 업로드 완료: uniqlo025
문서 업로드 완료: uniqlo026
문서 업로드 완료: uniqlo027
문서 업로드 완료: uniqlo028
문서 업로드 완료: uniqlo029
문서 업로드 완료: uniqlo030
문서 업로드 완료: uniqlo031
문서 업로드 완료: uniqlo032
문서 업로드 완료: uniqlo033
문서 업로드 완료: uniqlo034
문서 업로드 완료: uniqlo035
문서 업로드 완료: uniqlo036
문서 업로드 완료: uniqlo037
문서 업로드 완료: uniqlo038
문서 업로드 완료: uniqlo039
문서 업로드 완료: uniqlo040
문서 업로드 완료: uniqlo041
문서 업로드 완료: uniqlo042
문서 업로드 완료: uniqlo043
문서 업로드 완료: uniqlo044
문서 업로드 완료: uniqlo045
문서 업로드 완료: uniqlo046
문서 업로드 완료: uniqlo047
문서 업로드 완료: un

## 5. 자연어 쿼리로 검색 및 유사 문서 찾기

- 사용자의 질문을 임베딩으로 변환하고, Azure AI Search에서 벡터 유사도 기반으로 관련 문서를 검색합니다.

아래는 쿼리 임베딩 생성 및 검색 예시입니다.

In [1]:
import os
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery # VectorQuery에서 변경
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

load_dotenv()

openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
embedding_model = os.getenv("AZURE_OPENAI_DEPLOYMENT_EMBEDDING_NAME")
deployment_model = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")

client = AzureOpenAI(
    api_key=openai_key,
    api_version=openai_api_version,
    azure_endpoint=openai_endpoint
)
search_client = SearchClient(
    endpoint=search_endpoint, 
    index_name=index_name, 
    credential=AzureKeyCredential(search_key)
)

query_text = "남성용으로 여름에 시원하게 입을 수 있는 옷 추천해줘" # 사용자 질문
results = [] # results 변수를 try 블록 외부에서 초기화

try:
    # 질문을 임베딩으로 변환
    query_embedding = client.embeddings.create(
        input=query_text,
        model=embedding_model
    ).data[0].embedding

    # VectorizedQuery 객체 생성 (참조 코드에 따라 수정)
    vector_query = VectorizedQuery(
        vector=query_embedding, 
        k_nearest_neighbors=50, # 참조 코드에 따라 50으로 변경
        fields="embedding"  # 노트북 인덱스의 필드명 사용
    )

    # 검색 실행 (참조 코드에 따라 select 및 top 파라미터 추가/수정)
    search_results_iterator = search_client.search(
        search_text=None,  
        vector_queries=[vector_query],
        select=["id", "content", "metadata"], # 반환할 필드 선택
        top=3  # 상위 3개 결과 요청
    )
    results = list(search_results_iterator) # 이터레이터를 즉시 리스트로 변환하여 results에 저장

    # 결과 처리 (선택된 필드들을 출력하도록 수정)
    found_any_results = False
    for result_document in results: # 이제 results는 리스트입니다.
        found_any_results = True
        print("\n--- Search Result ---")
        print(f"ID: {result_document.get('id')}")
        # content 필드가 없을 경우를 대비해 .get() 사용 및 길이 제한
        content_snippet = result_document.get('content', '')
        print(f"Content: {content_snippet[:150]}...") 
        print(f"Metadata: {result_document.get('metadata')}")
        print("---------------------")
    
    if not found_any_results:
        print("\nNo documents found matching your query.")

except Exception as e: 
    print(f"오류 발생: {e}")



--- Search Result ---
ID: uniqlo023
Content: 이지 쇼트 팬츠 | 남성, 밴딩, 쿨링, 여름 필수템. 허리 밴딩과 쿨링 소재로 무더운 여름에도 편안하게 입을 수 있습니다. 다양한 상의와 매치해 캐주얼하게 연출할 수 있습니다....
Metadata: 유니클로, 팬츠, 쇼트, 남성, 쿨링, 여름
---------------------

--- Search Result ---
ID: uniqlo048
Content: 에어리즘 스트레치 쇼트 팬츠 | 남성, 쿨링, 여름 필수템. 신축성과 쿨링 기능으로 무더운 여름에도 쾌적하게 입을 수 있습니다. 다양한 상의와 매치해 캐주얼하게 연출할 수 있습니다....
Metadata: 유니클로, 팬츠, 쇼트, 남성, 쿨링, 여름
---------------------

--- Search Result ---
ID: uniqlo008
Content: 프리미엄 리넨 셔츠 | 남성, 시원한 린넨, 여름 인기. 통기성이 좋아 무더운 날씨에도 쾌적하게 입을 수 있습니다. 자연스러운 핏으로 세련된 스타일을 연출합니다....
Metadata: 유니클로, 셔츠, 린넨, 남성, 여름
---------------------


## 6. RAG(Retrieval Augmented Generation) 구현

5번 예제에서 검색된 문서를 활용하여 RAG 파이프라인을 구축할 수 있습니다. RAG는 검색된 정보를 바탕으로 LLM(Large Language Model)이 보다 정확하고 풍부한 답변을 생성하도록 돕는 기술입니다.

**RAG 구현 단계:**

1.  **사용자 질문 접수:** 사용자의 질문을 받습니다. (5번 예제의 `query_text`)
2.  **관련 문서 검색:**
    *   사용자 질문을 임베딩으로 변환합니다.
    *   Azure AI Search를 사용하여 관련성이 높은 문서를 검색합니다. (5번 예제의 `search_client.search(...)` 결과)
3.  **프롬프트 구성:**
    *   검색된 문서의 내용(예: `result_document['content']`)을 컨텍스트로 활용합니다.
    *   사용자의 원본 질문과 검색된 컨텍스트를 결합하여 LLM에 전달할 프롬프트를 생성합니다. 예를 들어, 다음과 같은 형식을 사용할 수 있습니다:
        ```
        다음 컨텍스트를 참고하여 질문에 답변해주세요.
        
        컨텍스트:
        [검색된 문서 1의 내용]
        [검색된 문서 2의 내용]
        ...
        
        질문: [사용자의 원본 질문]
        ```
4.  **LLM을 통한 답변 생성:**
    *   구성된 프롬프트를 Azure OpenAI의 언어 모델(예: `gpt-4.1-mini`, `gpt-4o-mini`)에 전달합니다.
    *   LLM은 제공된 컨텍스트를 기반으로 사용자의 질문에 대한 답변을 생성합니다.
5.  **답변 반환:** 생성된 답변을 사용자에게 제공합니다.

**구현 시 고려사항:**

*   **컨텍스트 길이:** LLM에 전달하는 컨텍스트의 길이는 모델의 토큰 제한을 고려해야 합니다. 필요시 검색된 문서의 내용을 요약하거나, 가장 관련성 높은 부분만 추출하여 사용합니다.
*   **프롬프트 엔지니어링:** LLM이 컨텍스트를 효과적으로 활용하고 원하는 답변을 생성하도록 프롬프트를 잘 설계하는 것이 중요합니다.
*   **반복 및 개선:** 검색 결과가 만족스럽지 않거나 LLM의 답변 품질이 낮을 경우, 검색 쿼리, 임베딩 모델, 프롬프트 등을 조정하며 반복적으로 개선합니다.

이 가이드를 바탕으로 5번에서 구현한 벡터 검색 결과를 활용하여 RAG 기반의 질의응답 시스템을 구축해볼 수 있습니다.

In [2]:
# (이전 셀에서 client, search_client, query_text, results 등이 정의되었다고 가정)

# 프롬프트 구성

try:
    # `results` 변수가 정의되지 않았다면 아래 접근 시 NameError 발생
    if results is None: # 추가된 체크: results가 None인지 확인
        raise ValueError("`results` 변수가 None입니다. 5번 '자연어 쿼리로 검색' 셀의 실행을 확인해주세요.")

    retrieved_context = "\n\n".join([doc.get('content', '') for doc in results])

    prompt = f"""다음 컨텍스트를 참고하여 질문에 답변해주세요.

컨텍스트:
{retrieved_context}

질문: {query_text}
"""

    print("--- 구성된 프롬프트 ---")
    print(prompt)
    print("---------------------")

    # LLM을 통한 답변 생성 (Azure OpenAI SDK 실제 호출)
    from openai import AzureOpenAI
    openai_client = AzureOpenAI(
        api_key=openai_key,
        api_version=openai_api_version,
        azure_endpoint=openai_endpoint
    )

    completion = openai_client.chat.completions.create(
        model=deployment_model,  # 사용자의 모델 배포명으로 필요시 변경
        messages=[
            {"role": "system", "content": "당신은 유용한 AI 어시스턴트입니다."},
            {"role": "user", "content": prompt}
        ]
    )
    generated_answer = completion.choices[0].message.content
    print(f"\nLLM 답변: {generated_answer}")

except Exception as e:
    print(f"오류 발생: {e}.\n이전 셀(특히 5번 '자연어 쿼리로 검색' 셀)의 실행 상태, results 변수, API 키 및 엔드포인트 설정을 확인해주세요.")

--- 구성된 프롬프트 ---
다음 컨텍스트를 참고하여 질문에 답변해주세요.

컨텍스트:
이지 쇼트 팬츠 | 남성, 밴딩, 쿨링, 여름 필수템. 허리 밴딩과 쿨링 소재로 무더운 여름에도 편안하게 입을 수 있습니다. 다양한 상의와 매치해 캐주얼하게 연출할 수 있습니다.

에어리즘 스트레치 쇼트 팬츠 | 남성, 쿨링, 여름 필수템. 신축성과 쿨링 기능으로 무더운 여름에도 쾌적하게 입을 수 있습니다. 다양한 상의와 매치해 캐주얼하게 연출할 수 있습니다.

프리미엄 리넨 셔츠 | 남성, 시원한 린넨, 여름 인기. 통기성이 좋아 무더운 날씨에도 쾌적하게 입을 수 있습니다. 자연스러운 핏으로 세련된 스타일을 연출합니다.

질문: 남성용으로 여름에 시원하게 입을 수 있는 옷 추천해줘

---------------------

LLM 답변: 여름에 남성용으로 시원하게 입을 수 있는 옷으로는 다음을 추천합니다.

1. 이지 쇼트 팬츠: 허리 밴딩과 쿨링 소재로 편안하고 시원하게 착용할 수 있어 무더운 여름에 적합합니다.

2. 에어리즘 스트레치 쇼트 팬츠: 신축성과 쿨링 기능이 있어 쾌적한 착용감을 제공합니다.

3. 프리미엄 리넨 셔츠: 통기성이 뛰어난 린넨 소재로 무더운 날씨에도 쾌적하며 자연스러운 핏으로 세련된 스타일을 연출할 수 있습니다.

이 제품들은 모두 여름에 캐주얼하게 매치하기 좋은 아이템입니다.


---

이 노트북에서는 JSON 데이터를 Azure AI Search에 업로드하고, OpenAI 임베딩을 활용해 벡터 검색을 구현하는 RAG 실습을 진행했습니다.

- Azure 리소스 준비 → JSON 업로드 및 인덱싱 → 임베딩 생성 및 저장 → 벡터 검색
- 실제 서비스 적용 시, 보안, 대용량 처리, 인덱스 관리 등 추가 고려가 필요합니다.

### 참고 자료
- [Azure AI Search 공식 문서](https://learn.microsoft.com/azure/search/)
- [Azure OpenAI Service 공식 문서](https://learn.microsoft.com/azure/cognitive-services/openai/)
